In [1]:
import numpy as np
import pandas as pd
import csv
import datetime

In [2]:
def createDF(infile_name):
    DF = pd.read_csv(infile_name, header=0, sep=',')
    return DF

In [3]:
#read data into pandas df
nfl_stats_2017 = createDF("nflstats2017.csv")

In [4]:
#Split Start into Date and Start Time
nfl_stats_2017[['Start','Start_Time']] = nfl_stats_2017['Start'].str.split(' ',expand=True)
nfl_stats_2017 = nfl_stats_2017.rename(index=str, columns = {"Start":"Date"})

#create Datetime values for Date & format
nfl_stats_2017['Date_DT'] = nfl_stats_2017['Date'].map(lambda x: datetime.datetime.strptime(x, '%m/%d/%Y')) 

#create keys for join
nfl_stats_2017['f_date'] = nfl_stats_2017['Date_DT'].map(lambda x: x.strftime('%Y%m%d'))
nfl_stats_2017 = nfl_stats_2017.assign(Key = lambda x: nfl_stats_2017.f_date+nfl_stats_2017.Home)



In [5]:
#drop unnecessary columns
nfl_stats_2017 = nfl_stats_2017.drop(['f_date'], axis = 1)

dslg_data = nfl_stats_2017.drop(['Season', 'Date', 'Week', 'APts', 'HPts', 'OverUnder',
       'VegasLine', 'AFD', 'AFum', 'AFumL', 'APY', 'APA', 'API', 'HPS', 'HPSY',
       'ARA', 'ARY', 'HFD', 'HFum', 'HFumL', 'HPY', 'HPA', 'HPI', 'HPS.1',
       'HPSY.1', 'HRA', 'HRY', 'Start_Time'], axis = 1)

dslg_data

,Away,Home,Date_DT,Key
0,KAN,NWE,2017-09-07,20170907NWE
1,NYJ,BUF,2017-09-10,20170910BUF
2,ARI,DET,2017-09-10,20170910DET
3,JAX,HOU,2017-09-10,20170910HOU
4,PIT,CLE,2017-09-10,20170910CLE
5,CAR,SFO,2017-09-10,20170910SFO
6,OAK,TEN,2017-09-10,20170910TEN
7,ATL,CHI,2017-09-10,20170910CHI
8,BAL,CIN,2017-09-10,20170910CIN
9,PHI,WAS,2017-09-10,20170910WAS


In [6]:
#create list of teams
teams = dslg_data['Away'].unique()

In [7]:
#create days between games played 

#create empty dataframes
Away_team = pd.DataFrame(columns = dslg_data.columns)
Home_team = pd.DataFrame(columns = dslg_data.columns)

for team in teams:
    #create team specific table 
    t_table = dslg_data.loc[(dslg_data['Away'] == team) | (dslg_data['Home'] == team)]
    
    #sort t_table by DT_date 
    t_table = t_table.sort_values(by = ['Date_DT'])
    
    #create boolean for Last Week Home Game -- week 1 starts as 1 for every team
    LW_HG = []

    for h in range(0,len(t_table['Home'])):
        if h == 0:
            val = 1
            LW_HG.append(val)  
        else:
            if t_table['Home'][h-1] == team:
                val = 1
            else:
                val = 0
            LW_HG.append(val)
            
    #create temp dataframe for LW_Home -- note column name is number 0
    LWHG = pd.DataFrame(data = LW_HG)
    
    #reset indexes, join tables, rename column
    t_table = t_table.reset_index(drop = True)
    t_table = t_table.join(LWHG)
    t_table = t_table.rename(index=str, columns = {0:"LW_HomeGame"})
    
    
    #create boolean for Last Week Away Game -- week 1 starts as 0 for every team
    LW_AG = []

    for a in range(0,len(t_table['Away'])):
        if a == 0:
            val = 0
            LW_AG.append(val)  
        else:
            if t_table['Away'][a-1] == team:
                val = 1
            else:
                val = 0
            LW_AG.append(val)
            
    #create temp dataframe for LW_Home -- note column name is number 0
    LWAG = pd.DataFrame(data = LW_AG)
    
    #reset indexes, join tables, rename column
    t_table = t_table.reset_index(drop = True)
    t_table = t_table.join(LWAG)
    t_table = t_table.rename(index=str, columns = {0:"LW_AwayGame"}) 
    
    
    #create Days Since Last Game 
    DT_Days_LastGame = []

    for i in range(0,len(t_table['Date_DT'])):
        if i == 0:
            #first week set to 14 for all teams to act as rest week
            val = t_table['Date_DT'][i] - (t_table['Date_DT'][i] - datetime.timedelta(days=14))
            DT_Days_LastGame.append(val)  
        else:
            val = t_table['Date_DT'][i] - t_table['Date_DT'][i-1]
            DT_Days_LastGame.append(val)
     
    
    #create integer values for days since last game
    Days_SLG = []
    
    for i in range(0,len(DT_Days_LastGame)):
        days = DT_Days_LastGame[i].total_seconds()/(60*60*24)
        Days_SLG.append(round(days))
    
    #create temp dataframe for Days_SLG -- note column name is number 0
    DSLG = pd.DataFrame(data = Days_SLG)
    
    #reset indexes, join tables, rename column
    t_table = t_table.reset_index(drop = True)
    t_table = t_table.join(DSLG)
    t_table = t_table.rename(index=str, columns = {0:"DaysSLG"})   
    
    
    #breakup into home games and away games. Create new or append to exisiting tables
    H_team = t_table.loc[t_table['Home'] == team]
    Home_team = Home_team.append(H_team,ignore_index=True)
    
    A_team = t_table.loc[t_table['Away'] == team]
    Away_team = Away_team.append(A_team,ignore_index=True)


In [8]:
#drop unnecessary columns Away table & rename columns
Away_team = Away_team.drop(['Away', 'Date_DT', 'Home'], axis = 1)
Away_team = Away_team.rename(index=str, columns = {'DaysSLG':"A_DaysSLG", 'LW_HomeGame':"A_LWHomeG",'LW_AwayGame': "A_LWAwayG"})

#drop unnecessary columns Home table & rename columns
Home_team = Home_team.drop(['Away', 'Date_DT', 'Home'], axis = 1)
Home_team = Home_team.rename(index=str, columns = {'DaysSLG':"H_DaysSLG", 'LW_HomeGame':"H_LWHomeG",'LW_AwayGame': "H_LWAwayG"})


In [9]:
#join tables
nfl_stats_2017 = nfl_stats_2017.join(Away_team.set_index('Key'), on='Key',how = 'left')
nfl_stats_2017 = nfl_stats_2017.join(Home_team.set_index('Key'), on='Key',how = 'left')

In [10]:
##Add in a new feature --Subtract:  H_LWHomeG - A_LWHomeG 

#A-LWAway + H-LWAway = 0 -- both travel
#A-LWAway + H-LWHome = 1 -- home team advantage
#A-LWHome + H-LWAway = -1 -- home team disadvantage 
#A-LWHome + H-LWHome = 0 -- both home

#create time zone diff for away team
nfl_stats_2017 = nfl_stats_2017.assign(H_LWGAdv = lambda x: nfl_stats_2017.H_LWHomeG - nfl_stats_2017.A_LWHomeG)

In [11]:
#check using NWE games
NWE = nfl_stats_2017.loc[(nfl_stats_2017['Away'] == 'NWE') | (nfl_stats_2017['Home'] == 'NWE')]
NWE = NWE.sort_values(by = ['Date_DT'])

NWE[['Date', 'Week', 'Away', 'Home', 'Date_DT', 'Key', 'A_DaysSLG',
       'A_LWAwayG', 'A_LWHomeG', 'H_DaysSLG', 'H_LWAwayG', 'H_LWHomeG']]

,Date,Week,Away,Home,Date_DT,Key,A_DaysSLG,A_LWAwayG,A_LWHomeG,H_DaysSLG,H_LWAwayG,H_LWHomeG
0,9/7/2017,1,KAN,NWE,2017-09-07,20170907NWE,14.0,0.0,1.0,14.0,0.0,1.0
26,9/17/2017,2,NWE,NOR,2017-09-17,20170917NOR,10.0,0.0,1.0,6.0,1.0,0.0
32,9/24/2017,3,HOU,NWE,2017-09-24,20170924NWE,10.0,1.0,0.0,7.0,1.0,0.0
59,10/1/2017,4,CAR,NWE,2017-10-01,20171001NWE,7.0,0.0,1.0,7.0,0.0,1.0
63,10/5/2017,5,NWE,TAM,2017-10-05,20171005TAM,4.0,0.0,1.0,4.0,0.0,1.0
86,10/15/2017,6,NWE,NYJ,2017-10-15,20171015NYJ,10.0,1.0,0.0,7.0,1.0,0.0
92,10/22/2017,7,ATL,NWE,2017-10-22,20171022NWE,7.0,0.0,1.0,7.0,1.0,0.0
114,10/29/2017,8,LAC,NWE,2017-10-29,20171029NWE,7.0,0.0,1.0,7.0,0.0,1.0
134,11/12/2017,10,NWE,DEN,2017-11-12,20171112DEN,14.0,0.0,1.0,7.0,1.0,0.0
154,11/19/2017,11,NWE,OAK,2017-11-19,20171119OAK,7.0,1.0,0.0,14.0,1.0,0.0


In [12]:
#Create table to CSV

#drop unnecessary columns
DSLG_2017 = nfl_stats_2017.drop(['Season', 'Date', 'Week', 'Away', 'Home', 'APts', 'HPts', 'OverUnder',
       'VegasLine', 'AFD', 'AFum', 'AFumL', 'APY', 'APA', 'API', 'HPS', 'HPSY',
       'ARA', 'ARY', 'HFD', 'HFum', 'HFumL', 'HPY', 'HPA', 'HPI', 'HPS.1',
       'HPSY.1', 'HRA', 'HRY', 'Start_Time', 'Date_DT'],axis=1)

#write table to csv
DSLG_2017.to_csv("DSLG_2017", encoding='utf-8', index=False)

DSLG_2017

,Key,A_DaysSLG,A_LWAwayG,A_LWHomeG,H_DaysSLG,H_LWAwayG,H_LWHomeG,H_LWGAdv
0,20170907NWE,14.0,0.0,1.0,14.0,0.0,1.0,0.0
1,20170910BUF,14.0,0.0,1.0,14.0,0.0,1.0,0.0
2,20170910DET,14.0,0.0,1.0,14.0,0.0,1.0,0.0
3,20170910HOU,14.0,0.0,1.0,14.0,0.0,1.0,0.0
4,20170910CLE,14.0,0.0,1.0,14.0,0.0,1.0,0.0
5,20170910SFO,14.0,0.0,1.0,14.0,0.0,1.0,0.0
6,20170910TEN,14.0,0.0,1.0,14.0,0.0,1.0,0.0
7,20170910CHI,14.0,0.0,1.0,14.0,0.0,1.0,0.0
8,20170910CIN,14.0,0.0,1.0,14.0,0.0,1.0,0.0
9,20170910WAS,14.0,0.0,1.0,14.0,0.0,1.0,0.0
